# Project Title
### Data Engineering Capstone Project

#### Project Summary
The project aims to leverage data engineering techniques to analyze and forecast tourism patterns and trends. It ingests and processes 2 data sets related to immigration, and weather detials. These features are further utilized to assist with tourism planning and enable accruate forecating of tourist arrivals. The value of such an itiative lies with the contribituon of economic development that tourism provides. Interest is in the number of immigrants for a country, on a month by month basis. Note this projects takes into account 2 stakeholders of the data. One that will be situated with analytical tasks, the other with forecasting.

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [68]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql.functions import col, count, isnan, when
from pyspark.sql import SparkSession
import sqlite3


In [12]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Step 1: Scope the Project and Gather Data

#### Scope 
<!-- Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc> -->

The purpose of the project is tourism planning and forecasting. Where relevant datasets can be used to develop predictive models. By analyzing historical immigration data, temperature patterns, city demographics, and airport details, these models can be leveraged to identify areas of opportunity and optimize tourism strategies. The clients considered in the scope of this project are government agencies, travel companies, and marketing iniatives.

[comment]: <> (This is a comment, it will not be included)

[//]: <> (This is also a comment.)

#### Describe and Gather Data 
<!-- Describe the data sets you're using. Where did it come from? What type of information is included? -->

I94 Immigration Data: 
- Source: https://www.trade.gov/national-travel-and-tourism-office
- Features: comphrehensive demographic and travel information

World Temperature Data: 
- Source: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data
- Features: time series of temperature segmented by country and city

<!-- U.S. City Demographic Data: 
- Source: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/
- Features: the demographics of all US cities and census-designated places (population >= 65,000)

Airport Code Table:
- Source: https://datahub.io/core/airport-codes#data
- Features: airport codes and corresponding cities -->

In [42]:
# Read in the data here
# airport = pd.read_csv('airport-codes_csv.csv')
# demo = pd.read_csv('us-cities-demographics.csv')


immi = pd.read_csv('immigration_data_sample.csv')
# i94 = pd.read_sas('I94_SAS_Labels_Descriptions.SAS')

fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temp = pd.read_csv(fname)



In [ ]:
# dsd = {}
# # for dataset in datasets:
#     dsd[dataset] = pd.read_sas(dataset+".sas7bdat", encoding = 'latin-1')

In [4]:
immi.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [5]:
immi.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [6]:
temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [43]:
# Define the path to your data file
data_file_path = "country_name.txt"

# Read the data file
with open(data_file_path, "r") as file:
    lines = file.readlines()

# Create a dictionary to store the mappings
country_mapping = {}

# Process each line and extract the code and name
for line in lines:
    code, name = line.strip().split(" = ")
    code = int(code)
    name = name.strip("'")
    country_mapping[code] = name
# print(country_mapping)
country_name = pd.DataFrame(list(country_mapping.items()), columns=['Code', 'Country'])
country_name.head()

,Code,Country
0,582,"'MEXICO Air Sea, and Not Reported (I-94, no l..."
1,236,'AFGHANISTAN
2,101,'ALBANIA
3,316,'ALGERIA
4,102,'ANDORRA


In [64]:
# import os
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

immi = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

# folder_path = '../../data/18-83510-I94-Data-2016/'

# Get a list files
# file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.sas7bdat')]
# immi = spark.read.format("com.github.saurfang.sas.spark").load(file_paths)
immi.show(n = 5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [35]:
file_path = '../../data2/GlobalLandTemperaturesByCity.csv'
# temp = spark.read.csv(file_path, header=True, inferSchema=True)

In [37]:
# temp.show(n = 5)

+-------------------+------------------+-----------------------------+-----+-------+--------+---------+
|                 dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+-------------------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01 00:00:00|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01 00:00:00|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01 00:00:00|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01 00:00:00|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01 00:00:00|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+-------------------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



In [40]:
# country_name = spark.createDataFrame(country_name)
# country_name.show(n = 5)

+----+--------------------+
|Code|             Country|
+----+--------------------+
| 582| 'MEXICO Air Sea,...|
| 236|        'AFGHANISTAN|
| 101|            'ALBANIA|
| 316|            'ALGERIA|
| 102|            'ANDORRA|
+----+--------------------+
only showing top 5 rows



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [41]:
using the sample data/pandas
immi.isna().sum()/immi.shape[0]*100
immi.dtypes
len(immi)-len(immi.drop_duplicates())

# using spark
# missing_values_count = immi.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in immi.columns])
# missing_values_count.show()

# duplicate_count = immi.groupBy(immi.columns).count().where(col("count") > 1)
# duplicate_count.show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|  occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender| insnum|airline|admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|    0|    0|     0|     0|     0|      0|      0|    239| 152592| 142457|   802|      0|    0|       1| 1881250|3088187|    238| 138429|3095921| 138429|    802|    477|414269|2982605|  83627|     0|19549|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+---

NameError: name 'immigration_df' is not defined

In [29]:
immi.shape[0]
immi.isna().sum()/immi.shape[0]*100
immi.dtypes
len(immi)-len(immi.drop_duplicates())

1000

Unnamed: 0      0.0
cicid           0.0
i94yr           0.0
i94mon          0.0
i94cit          0.0
i94res          0.0
i94port         0.0
arrdate         0.0
i94mode         0.0
i94addr         5.9
depdate         4.9
i94bir          0.0
i94visa         0.0
count           0.0
dtadfile        0.0
visapost       61.8
occup          99.6
entdepa         0.0
entdepd         4.6
entdepu       100.0
matflag         4.6
biryear         0.0
dtaddto         0.0
gender         14.1
insnum         96.5
airline         3.3
admnum          0.0
fltno           0.8
visatype        0.0
dtype: float64

Unnamed: 0      int64
cicid         float64
i94yr         float64
i94mon        float64
i94cit        float64
i94res        float64
i94port        object
arrdate       float64
i94mode       float64
i94addr        object
depdate       float64
i94bir        float64
i94visa       float64
count         float64
dtadfile        int64
visapost       object
occup          object
entdepa        object
entdepd        object
entdepu       float64
matflag        object
biryear       float64
dtaddto        object
gender         object
insnum        float64
airline        object
admnum        float64
fltno          object
visatype       object
dtype: object

0

In [15]:
# using pandas
temp.shape[0]
temp.isna().sum()/temp.shape[0]*100
temp.dtypes
len(temp)-len(temp.drop_duplicates())

# using spark

# missing_values_count = temp.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in temp.columns])
# missing_values_count.show()

# duplicate_count = temp.groupBy(temp.columns).count().where(col("count") > 1)
# duplicate_count.show()

8599212

dt                               0.000000
AverageTemperature               4.234458
AverageTemperatureUncertainty    4.234458
City                             0.000000
Country                          0.000000
Latitude                         0.000000
Longitude                        0.000000
dtype: float64

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

0

<!-- For each of the data sets we took a look at the percentage of missing values for each column.

Since we do not yet know which columns will be used for joining the data. We will proceed by droping rows that have more than 30% missing values. Duplicates can be removed. -->


The 2 columns of most interest for joining the data set have no missing values in the immigration data. (ie,m i94cit, i94mon). Therefore, we dont have much cause for concern and all rows can be kept. We have that some of the country codes are of a float type. For analytical and forecasting purposes this would be problematic. There are no duplicates found in the data set


Note that, the temperature data is already aggregated by month, and there exists some missing values. Temperature ius presumed to be a significant feature in this data set. So having 4% missing values is not ideal. Since it is only 4%, we can either remove those rows (losing insight) or fill in the missing data. To proceed, we will drop the rows. Although, if we did want to fill by mean, we would have to consider a partition for each country ordered by data to ensure quality temperatures. 


Also, the temperature data set only contains data up until 2013, wheras the forecasting data goes up until 2016.

In [55]:
# Performing cleaning tasks here
# immi2 = immi.select("i94yr", "i94mon", "i94cit", "i94res", "i94port", "i94mode", "i94addr", "i94visa") \

immi2 = immi[["i94yr", "i94mon", "i94cit", "i94res", "i94port", "i94mode", "i94addr", "i94visa"]]

temp2 = temp.dropna()




### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

(this includes Data dictionary )
<!-- Map out the conceptual data model and explain why you chose that model -->

Immigration: Fact table; captures the primary facts
- i94yr: Year of immigration
- i94mon: Month of immigration
- i94cit: Origin country code
- i94res: Residence country code
- i94port: Port of entry code
- i94mode: Mode of transportation code
- i94addr: US state code of destination
- i94visa: Visa category code
- count: Count of immigrants

Temperature: Dimension table; additional details
- Country: Country name
- dt: Date
- AverageTemperature: Average temperature

Country: Dimension table; additional details
- CountryCode: Country code
- CountryName: Country name

A star schema representation (relational data model) utilizes SQL queries that provide a foundation for performing analytical tasks, explorations (descriptive) and predictive analytics for different use-cases. Further more, it simple, flexible and scalable, considers relationships between data, promots consistency and data integrity, and is easy to work with. 

#### 3.2 Mapping Out Data Pipelines
<!-- List the steps necessary to pipeline the data into the chosen data model -->
1. Retrieve data (data extraction)
2. Data Preprocessing 
3. Aggregate and transform data 


For ongoing analysis we would considering the following additional steps

4. Create tables in data base 
5. Load data to database 
6. Data validation
7. Data storage 
8. Storage optimization and maintenance
9. Periodic updates 



### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [56]:
temp2['dt'] = pd.to_datetime(temp2['dt'])
# temp2.dtypes

temp2['year'] = temp2['dt'].dt.year
temp2['month'] = temp2['dt'].dt.month

temp2.head()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E,1743,11
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E,1744,4
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E,1744,5
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E,1744,6
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E,1744,7


In [60]:
temp2.year.max()


2013

In [52]:
immi2['i94yr'] = immi2['i94yr'].astype(int)
immi2['i94mon'] = immi2['i94mon'].astype(int)
immi2.head()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,i94yr,i94mon,i94cit,i94res,i94port,i94mode,i94addr,i94visa
0,2016,4,209.0,209.0,HHW,1.0,HI,2.0
1,2016,4,582.0,582.0,MCA,1.0,TX,2.0
2,2016,4,148.0,112.0,OGG,1.0,FL,2.0
3,2016,4,297.0,297.0,LOS,1.0,CA,2.0
4,2016,4,111.0,111.0,CHM,3.0,NY,2.0


In [69]:
conn = sqlite3.connect('capstone_project.db')  

In [72]:
conn.execute('''
    CREATE TABLE immigration_data (
        id INTEGER PRIMARY KEY,
        country TEXT,
        date DATE,
        visitor_type TEXT,
        visitor_count INTEGER,
        visa_type TEXT,
        duration_of_stay INTEGER,
        port_of_entry TEXT
    )
''')

conn.execute('''
    CREATE TABLE country_names (
        country_code TEXT PRIMARY KEY,
        country_name TEXT
    )
''')

# Create the temperature_data table
conn.execute('''
    CREATE TABLE temperature_data (
        id INTEGER PRIMARY KEY,
        country_code TEXT,
        date DATE,
        average_temperature FLOAT,
        temperature_uncertainty FLOAT
    )
''')



conn.commit()


In [74]:
# Step 3: Upload the DataFrame to the database
immi3 = immi2.reset_index()
immi3.head()
immi3.to_sql('immigration_data', conn, if_exists='replace', index=False)


,index,i94yr,i94mon,i94cit,i94res,i94port,i94mode,i94addr,i94visa
0,0,2016.0,4.0,209.0,209.0,HHW,1.0,HI,2.0
1,1,2016.0,4.0,582.0,582.0,MCA,1.0,TX,2.0
2,2,2016.0,4.0,148.0,112.0,OGG,1.0,FL,2.0
3,3,2016.0,4.0,297.0,297.0,LOS,1.0,CA,2.0
4,4,2016.0,4.0,111.0,111.0,CHM,3.0,NY,2.0


In [77]:
temp3 = temp2.reset_index()
temp3 = temp3[['index', 'dt', 'Country', 'AverageTemperature', 'AverageTemperatureUncertainty']]
temp3.head()

temp3.to_sql('temperature_data', conn, if_exists='replace', index=False)


,index,dt,Country,AverageTemperature,AverageTemperatureUncertainty
0,0,1743-11-01,Denmark,6.068,1.737
1,5,1744-04-01,Denmark,5.788,3.624
2,6,1744-05-01,Denmark,10.644,1.283
3,7,1744-06-01,Denmark,14.051,1.347
4,8,1744-07-01,Denmark,16.082,1.396


In [75]:
country_name.to_sql('country_names', conn, if_exists='replace', index=False)


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [85]:
# Perform quality checks here

immi_df = pd.read_sql_query('SELECT * FROM immigration_data', conn)
print(immi_df.index.is_unique)


temp_df = pd.read_sql_query('SELECT * FROM temperature_data', conn)
print(immi_df.index.is_unique)


country_df = pd.read_sql_query('SELECT * FROM country_names', conn)
print(country_df.Code.is_unique)

    
    



True
True
True


#### 4.3 Data dictionary 
<!-- Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file. -->


Country Names:

- country_code (Text): The unique code representing a country. It is sourced from the original country code dataset.
- country_name (Text): The name of the country corresponding to the country code. It is sourced from the original country name dataset.

Temperature Data Table:

- country_code (Text): The unique code representing a country. It is sourced from the original temperature dataset.
- year (Integer): The year for which the temperature data is recorded. It is sourced from the original temperature dataset.
- month (Integer): The month for which the temperature data is recorded. It is extracted from the original date field in the temperature dataset.
- temperature (Float): The temperature value recorded for the specific country, year, and month. It is sourced from the original temperature dataset.

Immigration Data Table:

- country_code (Text): The unique code representing a country. It is sourced from the original immigration dataset.
- year (Integer): The year for which the immigration data is recorded. It is sourced from the original immigration dataset.
- month (Integer): The month for which the immigration data is recorded. It is sourced from the original immigration dataset.
<!-- - count (Integer): The count of immigration events for the specific country, year, and month. It is sourced from the original immigration dataset. -->

#### Step 5: Complete Project Write Up
<!-- * Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people. -->

**Choice of tools**
Python: Python is a versatile and widely-used programming language with rich libraries and frameworks for data processing, analysis, and manipulation. It provides a large ecosystem of tools that make it well-suited for tasks like data cleaning, transformation, and analysis.

Pandas: 
- data manipulation library in Python that provides high-performance
- easy-to-use data structures and data analysis tools
- efficient data handling capabilities; such as data filtering, aggregation, merging, and transformation

SQL
- relational database management system that is easy to set up and use.
- suitable for local development and testing purposes where there is no need for complex database infrastructure. 
- good compatibility with Python and supports widely known SQL queries.

**Frequency of data update**
- Country data: is static and does not require updating
- Immigration data: Update monthly or quarterly, depending the availability. Since the analysis would like to have it has granular as possible
- Temperature: update daily to get the latest trends for quality decision-making

**Other Considerations**
 * The data was increased by 100x.
    - Switch to distributed data processing framework like Apache Spark. Since Spark can handle large-scale data and perform distributed processing to achieve scalability and performance.
    
* The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - Automate the data pipeline to ensure it runs at a specific time every day, preferably during off-peak hours to minimize disruption.
    - Utilize scheduling tools like cron jobs or job schedulers (e.g., Apache Airflow) to manage and trigger the data pipeline at the desired time.
 
* The database needed to be accessed by 100+ people.
    - Optimize query performance and scaling up the infrasture to handle cocurrent connections and query loads
    - Use Redis to optimize performance and reduce database load


 
 
 